In [102]:
import cv2
import numpy as np

BYTEORDER = 'little'

In [103]:
# Generate end marker
i0, i1 = 0, 1
b0, b1 = i0.to_bytes(1, byteorder=BYTEORDER, signed=False), i1.to_bytes(1, byteorder=BYTEORDER, signed=False)
end_marker = i0.to_bytes(7, byteorder=BYTEORDER, signed=False) + i1.to_bytes(1, byteorder=BYTEORDER, signed=False)

In [104]:
def index_hash (pixel: np.ndarray):
    if pixel.shape == 3:
        r, g, b, a = pixel
        index_position = (r * 3 + g * 5 + b * 7 + a * 11) % 64
    else:
        r, g, b = pixel
        index_position = (r * 3 + g * 5 + b * 7) % 64
    return int(index_position)

In [126]:
path = 'src/IMG_0036-v3-duplicate-downres.jpeg'

img = cv2.imread(path)
height, width, channels = img.shape
colorspace = 0  # srgb

if channels == 3:
    pixel_prev = np.array([0, 0, 0])
elif channels == 4:
    pixel_prev = np.array([0, 0, 0, 255])
else: print('error: wrong number of channels')

img_line = img.reshape((-1, 3))
print(f'Estimated processing time: {round(img_line.shape[0]/6000)}s')

Estimated processing time: 40s


## Encoding

In [ ]:
# Generate header
width_b = width.to_bytes(4, byteorder=BYTEORDER, signed=False)
height_b = height.to_bytes(4, byteorder=BYTEORDER, signed=False)
channels_b = channels.to_bytes(1, byteorder=BYTEORDER, signed=False)
colorspace_b = colorspace.to_bytes(1, byteorder=BYTEORDER, signed=False)

header = b'qoif' + width_b + height_b + channels_b + colorspace_b

In [121]:
bytes = b''

run_len = 0
run = False
memory = dict()
for pixel in img_line:

    # Unpack pixel
    if channels == 3:
        r, g, b = pixel
        r_prev, g_prev, b_prev = pixel_prev
        a, a_prev = 255, 255
    elif channels == 4:
        r, g, b, a = pixel
        r_prev, g_prev, b_prev, a_prev = pixel_prev


    # Run-length encoding
    if (pixel == pixel_prev).all() and run_len < 63:
        # print('run')
        run_len += 1
        run = True


    else:
        # End run-length
        if run == True:
            tag = b1 + b1
            val = run_len.to_bytes(6, byteorder=BYTEORDER, signed=True)
            bytes += (tag + val)  # write
            run_len = 0
            run = False

        # Memory search
        flag = False
        for key in memory.keys():
            if (memory[key] == pixel).all():
                # print('from memory')
                tag = b0 + b0
                val = key.to_bytes(6, byteorder=BYTEORDER, signed=False)
                flag = True
                break
        
        if flag == False:
            
            dr = int(r) - int(r_prev)
            dg = int(g) - int(g_prev)
            db = int(b) - int(b_prev)
            da = int(a) - int(a_prev)

            # Diff
            if dr in range(-2, 2) and dg in range(-2, 2) and db in range(-2, 2) and da == 0:
                # print('smalldif')
                tag = b0 + b1
                val = dr.to_bytes(2, byteorder=BYTEORDER, signed=True) + dg.to_bytes(2, byteorder=BYTEORDER, signed=True) + db.to_bytes(2, byteorder=BYTEORDER, signed=True)
            
            # Luma
            else:
                dr_dg = dr - dg
                db_dg = db - dg
                
                if dg in range(-32, 32) and dr_dg in range(-8, 8) and db_dg in range(-8, 8) and da == 0:
                    # print('bigdif')
                    tag = b1 + b0
                    val = dr.to_bytes(6, byteorder=BYTEORDER, signed=True) + dr_dg.to_bytes(4, byteorder=BYTEORDER, signed=True) + db_dg.to_bytes(4, byteorder=BYTEORDER, signed=True)

                # New pixel
                else:
                    # print('new')
                    if channels == 3:
                        tag = i1.to_bytes(7, byteorder=BYTEORDER, signed=False) + i0.to_bytes(1, byteorder=BYTEORDER, signed=False)
                        val = int(r).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(g).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(b).to_bytes(8, byteorder=BYTEORDER, signed=False)
                    elif channels == 4:
                        tag = i1.to_bytes(8, byteorder=BYTEORDER, signed=False)
                        val = int(r).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(g).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(b).to_bytes(8, byteorder=BYTEORDER, signed=False) + int(a).to_bytes(8, byteorder=BYTEORDER, signed=False)

        # Write
        bytes += (tag + val)

    # Add pixel to memory
    memory[index_hash(pixel)] = pixel
    pixel_prev = pixel.copy()

In [116]:
with open('out.bin', 'wb') as out_bin:
    out_bin.write(header)
    out_bin.write(bytes)
    out_bin.write(end_marker)


## Decoding

In [151]:
with open('out.bin', 'rb') as in_bin:
    img_bytes = in_bin.read()

header = img_bytes[:14]
bytes = img_bytes[14:-8]
# end_marker = img_bytes[-8:]

In [ ]:
### TODO: READ NB OF CHANNELS

if channels == 3:
    pixel_prev = np.array([0, 0, 0])
elif channels == 4:
    pixel_prev = np.array([0, 0, 0, 255])
else: print('error: wrong number of channels')

In [206]:
nbytes = len(bytes)
if nbytes % 8 != 0: print('corrupted file')
# for i in range(0, nbytes, 8):

if channels == 3:
    img_line = np.empty((0, 3), dtype=np.uint8)
elif channels == 4:
    img_line = np.empty((0, 4), dtype=np.uint8)

memory = dict()
i = 0
while i < nbytes - 8:
    chunk = bytes[i:i+8]
    
    # Original value (RGB)
    if channels == 3 and chunk == b'\x01\x01\x01\x01\x01\x01\x01\x00':
        r = bytes[i+8:i+16]
        g = bytes[i+16:i+24]
        b = bytes[i+24:i+32]
        r_int = int.from_bytes(r, byteorder=BYTEORDER, signed=False)
        g_int = int.from_bytes(g, byteorder=BYTEORDER, signed=False)
        b_int = int.from_bytes(b, byteorder=BYTEORDER, signed=False)
        pixel = np.array([r, g, b])
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        i += 24
    # Original value (RGBA)
    elif channels == 4 and chunk == b'\x01\x01\x01\x01\x01\x01\x01\x01':
        r = bytes[i+8:i+16]
        g = bytes[i+16:i+24]
        b = bytes[i+24:i+32]
        a = bytes[i+32:i+40]
        r_int = int.from_bytes(r, byteorder=BYTEORDER, signed=False)
        g_int = int.from_bytes(g, byteorder=BYTEORDER, signed=False)
        b_int = int.from_bytes(b, byteorder=BYTEORDER, signed=False)
        a_int = int.from_bytes(a, byteorder=BYTEORDER, signed=False)
        pixel = np.array([r, g, b, a])
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        i += 32

    # Big dif
    elif chunk[:2] == b'\x01\x00':
        dg = int.from_bytes(chunk[2:], byteorder=BYTEORDER, signed=True)
        dr = int.from_bytes(bytes[i+8:i+12], byteorder=BYTEORDER, signed=True) + dg
        db = int.from_bytes(bytes[i+12:i+16], byteorder=BYTEORDER, signed=True) + dg
        if channels == 3: pixel_diff = np.array([dr, dg, db])
        elif channels == 4: pixel_diff = np.array([dr, dg, db, 0])
        pixel = pixel_prev.copy() + pixel_diff
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))
        i += 8
    
    # Small dif
    elif chunk[:2] == b'\x00\x01':
        dr = int.from_bytes(chunk[2:4], byteorder=BYTEORDER, signed=True)
        dg = int.from_bytes(chunk[4:6], byteorder=BYTEORDER, signed=True)
        db = int.from_bytes(chunk[6:8], byteorder=BYTEORDER, signed=True)
        if channels == 3: pixel_diff = np.array([dr, dg, db])
        elif channels == 4: pixel_diff = np.array([dr, dg, db, 0])
        pixel = pixel_prev.copy() + pixel_diff
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))

    # Index
    elif chunk[:2] == b'\x00\x00':
        index = int.from_bytes(chunk[2:], byteorder=BYTEORDER, signed=False)
        pixel = memory[index].copy()
        img_line = np.concatenate((img_line, np.expand_dims(pixel, axis=0)))

    # Run
    elif chunk[:2] == b'\x01\x01':
        run_len = int.from_bytes(chunk[2:], byteorder=BYTEORDER, signed=False)
        pixels = np.full((run_len, channels), pixel_prev)
        img_line = np.concatenate((img_line, pixels))
    
    # Add pixel to memory
    memory[index_hash(pixel)] = pixel
    pixel_prev = pixel.copy()

    i += 8

print(img_line)

[[124   0   0]
 [250   0   0]
 [258   9   5]
 [271  16  17]
 [412  16  17]
 [414  21  22]
 [418  25  26]
 [565  25  26]
 [570  33  34]]


In [187]:
u = np.array([1, 2, 3])
v = np.array([0, 0, 0])
print(v.shape)

a = np.empty((0, 3), dtype=np.uint8)
print(a)
b = np.full((10, 3), u)

(3,)
[]


In [205]:
l = np.concatenate((a, b))

In [204]:
np.concatenate((l, np.expand_dims(v, axis=0)))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [0, 0, 0]])